In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import trange
import random
import math
from scipy import interp
import statistics 
import os
from tqdm import tqdm

from tcrmodels.ergo2.model import ERGO2
from tcrmodels.nettcr2.model import NetTCR2
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, auc

from matplotlib import collections
from matplotlib import colors
from numpy.random import normal

In [7]:
metrics = [
    'AUROC',
    'Accuracy',
    'Recall',
    'Precision',
    'F1 score',
    'AUPR'
]

def pr_auc(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall, precision)
    return pr_auc

def get_scores(y_true, y_prob, y_pred):
    """
    Compute a df with all classification metrics and respective scores.
    """
    
    scores = [
        roc_auc_score(y_true, y_prob),
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        f1_score(y_true, y_pred),
        pr_auc(y_true, y_prob)
    ]
    
    df = pd.DataFrame(data={'score': scores, 'metrics': metrics})
    return df

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)

In [9]:
login = os.getlogin( )
DATA_BASE = f"/home/{login}/Git/tc-hard/tc-hard-data/"
RESULTS_BASE = f"/home/{login}/Git/tc-hard/notebooks/notebooks.classification/results/"

In [10]:
import pickle as p

with open(DATA_BASE + "ergo2-paper/mcpas_train_samples.pickle", "rb") as f:
    mcpas_train = pd.DataFrame(p.load(f))

    
with open(DATA_BASE + "ergo2-paper/mcpas_test_samples.pickle", "rb") as f:
    mcpas_test = pd.DataFrame(p.load(f))

mcpas = pd.concat([mcpas_train, mcpas_test]).reset_index(drop=True)

mcpas = mcpas[["tcrb", "peptide", "sign"]].drop_duplicates(subset=["tcrb", "peptide"], keep=False).dropna().reset_index(drop=True)
print('pos:', len(mcpas[mcpas['sign']==1]))
print('neg:', len(mcpas[mcpas['sign']==0]))

pos: 8612
neg: 47630


In [11]:
import pickle as p

with open(DATA_BASE + "ergo2-paper/vdjdb_train_samples.pickle", "rb") as f:
    vdjdb_train = pd.DataFrame(p.load(f))

    
with open(DATA_BASE + "ergo2-paper/vdjdb_test_samples.pickle", "rb") as f:
    vdjdb_test = pd.DataFrame(p.load(f))

vdjdb = pd.concat([vdjdb_train, vdjdb_test]).reset_index(drop=True)

vdjdb = vdjdb[["tcrb", "peptide", "sign"]].drop_duplicates(subset=["tcrb", "peptide"], keep=False).dropna().reset_index(drop=True)
print('pos:', len(vdjdb[vdjdb['sign']==1]))
print('neg:', len(vdjdb[vdjdb['sign']==0]))

pos: 17534
neg: 106093


# NetTCR-2.0

In [14]:
results_nettcr2 = []

df_train = mcpas.copy()
df_test = vdjdb.copy()

max_pep_len = max(df_train["peptide"].str.len().max(), df_test["peptide"].str.len().max())
max_cdr3b_len = max(df_train["tcrb"].str.len().max(), df_test["tcrb"].str.len().max())


model = NetTCR2(
    architecture="b", 
    single_chain_column='tcrb',
    peptide_column='peptide',
    label_column='sign',
    max_pep_len=max_pep_len, 
    max_cdr3_len=max_cdr3b_len
)
model.train(df_train, epochs=1000);

prediction_df = model.test(df_test)

scores_df = get_scores(
    y_true=prediction_df['sign'].to_numpy(), 
    y_prob=prediction_df['prediction'].to_numpy(),
    y_pred=prediction_df['prediction'].to_numpy().round(),
)
results_nettcr2.append(scores_df)
df_test['prediction'] = prediction_df['prediction']
df_test.to_csv(RESULTS_BASE+f"nettcr2.mcpas2vdjdb.csv", index=False)

Epoch 1/1000
56242/56242 [==============================] - 4s 67us/step - loss: 0.4266
Epoch 2/1000
56242/56242 [==============================] - 3s 48us/step - loss: 0.4245
Epoch 3/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.4236
Epoch 4/1000
56242/56242 [==============================] - 3s 45us/step - loss: 0.4229
Epoch 5/1000
56242/56242 [==============================] - 3s 45us/step - loss: 0.4224
Epoch 6/1000
56242/56242 [==============================] - 3s 45us/step - loss: 0.4220
Epoch 7/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.4215
Epoch 8/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.4209
Epoch 9/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.4196
Epoch 10/1000
56242/56242 [==============================] - 3s 46us/step - loss: 0.4180
Epoch 11/1000
56242/56242 [==============================] - 2s 44us/step - loss: 0.4161
Epoch 12/1000
56242/56242 [===

56242/56242 [==============================] - 2s 42us/step - loss: 0.2600
Epoch 94/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2596
Epoch 95/1000
56242/56242 [==============================] - 2s 44us/step - loss: 0.2593
Epoch 96/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.2580
Epoch 97/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2572
Epoch 98/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2564
Epoch 99/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2555
Epoch 100/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2559
Epoch 101/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2542
Epoch 102/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.2539
Epoch 103/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.2530
Epoch 104/1000
56242/56242 [===

56242/56242 [==============================] - 2s 42us/step - loss: 0.1732
Epoch 276/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1725
Epoch 277/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1734
Epoch 278/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1731
Epoch 279/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1719
Epoch 280/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1722
Epoch 281/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1715
Epoch 282/1000
56242/56242 [==============================] - 3s 47us/step - loss: 0.1707
Epoch 283/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1710
Epoch 284/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1706
Epoch 285/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1698
Epoch 286/1000
56242/5624

56242/56242 [==============================] - 2s 43us/step - loss: 0.1298
Epoch 458/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1306
Epoch 459/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1294
Epoch 460/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1299
Epoch 461/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1294
Epoch 462/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1298
Epoch 463/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1287
Epoch 464/1000
56242/56242 [==============================] - 3s 45us/step - loss: 0.1291
Epoch 465/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1280
Epoch 466/1000
56242/56242 [==============================] - 3s 45us/step - loss: 0.1288
Epoch 467/1000
56242/56242 [==============================] - 2s 43us/step - loss: 0.1290
Epoch 468/1000
56242/5624

56242/56242 [==============================] - 2s 41us/step - loss: 0.1048
Epoch 640/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1046
Epoch 641/1000
56242/56242 [==============================] - 3s 45us/step - loss: 0.1038
Epoch 642/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1040
Epoch 643/1000
56242/56242 [==============================] - 2s 42us/step - loss: 0.1044
Epoch 644/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1042
Epoch 645/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1039
Epoch 646/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1043
Epoch 647/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1038
Epoch 648/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1034
Epoch 649/1000
56242/56242 [==============================] - 2s 41us/step - loss: 0.1040
Epoch 650/1000
56242/5624

In [19]:
def make_ergo_train_df(df):    
    # the ERGO II implementation expected the following columns to be preset in the dataframe
    # even if they are not used
    df['va'] = pd.NA
    df['vb'] = pd.NA
    df['ja'] = pd.NA
    df['jb'] = pd.NA
    df['t_cell_type'] = pd.NA
    df['protein'] = pd.NA
    df['mhc'] = pd.NA

    # using "UNK" for identifier of missing CDR3α
    df['tcra'] = "UNK"
    
    df['tcrb'] = df['tcrb'].str.replace('O','X')
    df['peptide'] = df['peptide'].str.replace('O','X')

    return df

def make_ergo_test_df(df):    
    map_keys = {
    'tcrb': 'TRB',
    'peptide': 'Peptide',
    'sign': 'sign'
    }
    df = df.rename(columns={c: map_keys[c] for c in df.columns})

    # the ERGO II implementation expected the following columns to be preset in the dataframe
    # even if they are not used
    df['TRAV'] = pd.NA
    df['TRBV'] = pd.NA
    df['TRAJ'] = pd.NA
    df['TRBJ'] = pd.NA
    df['T-Cell-Type'] = pd.NA
    df['Protein'] = pd.NA
    df['MHC'] = pd.NA

    # using "UNK" for identifier of missing CDR3α
    df['TRA'] = "UNK"
    df['TRB'] = df['TRB'].str.replace('O','X')
    df['Peptide'] = df['Peptide'].str.replace('O','X')

    return df

In [20]:
results_ergo2 = []

df_train = make_ergo_train_df(mcpas.copy())
df_test = make_ergo_test_df(vdjdb.copy())

model = ERGO2(
    gpu=[0],
    use_alpha=False,
    random_seed=0,
    train_val_ratio=.2,
)
model.train(df_train, epochs=1000)
prediction_df = model.test(df_test)

scores_df = get_scores(
    y_true=prediction_df['sign'].to_numpy(), 
    y_prob=prediction_df['prediction'].to_numpy(),
    y_pred=prediction_df['prediction'].to_numpy().round(),
)
results_ergo2.append(scores_df)
df_test['prediction'] = prediction_df['prediction']
df_test.to_csv(RESULTS_BASE+f"ergo2.mcpas2vdjdb.csv", index=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name          | Type         | Params
-----------------------------------------------
0 | tcrb_encoder  | LSTM_Encoder | 3 M   
1 | pep_encoder   | LSTM_Encoder | 3 M   
2 | hidden_layer1 | Linear       | 31 K  
3 | relu          | LeakyReLU    | 0     
4 | o

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]